In [1]:
import os
import sys
import timeit

from constants import *
from util import *
from analysis import *
from classifier import *

import numpy

import theano
import theano.tensor as T
from theano.tensor.signal import downsample
from theano.tensor.nnet import conv2d

from logistic_sgd import LogisticRegression, load_data
from mlp import HiddenLayer


from sklearn.cross_validation import train_test_split

%load_ext autoreload
%autoreload 2


In [2]:
class LeNetConvPoolLayer(object):
    """Pool Layer of a convolutional network """

    def __init__(self, rng, input, filter_shape, image_shape, poolsize=(2, 2)):
        """
        Allocate a LeNetConvPoolLayer with shared variable internal parameters.

        :type rng: numpy.random.RandomState
        :param rng: a random number generator used to initialize weights

        :type input: theano.tensor.dtensor4
        :param input: symbolic image tensor, of shape image_shape

        :type filter_shape: tuple or list of length 4
        :param filter_shape: (number of filters, num input feature maps,
                              filter height, filter width)

        :type image_shape: tuple or list of length 4
        :param image_shape: (batch size, num input feature maps,
                             image height, image width)

        :type poolsize: tuple or list of length 2
        :param poolsize: the downsampling (pooling) factor (#rows, #cols)
        """

        assert image_shape[1] == filter_shape[1]
        self.input = input

        # there are "num input feature maps * filter height * filter width"
        # inputs to each hidden unit
        fan_in = numpy.prod(filter_shape[1:])
        # each unit in the lower layer receives a gradient from:
        # "num output feature maps * filter height * filter width" /
        #   pooling size
        fan_out = (filter_shape[0] * numpy.prod(filter_shape[2:]) /
                   numpy.prod(poolsize))
        # initialize weights with random weights
        W_bound = numpy.sqrt(6. / (fan_in + fan_out))
        self.W = theano.shared(
            numpy.asarray(
                rng.uniform(low=-W_bound, high=W_bound, size=filter_shape),
                dtype=theano.config.floatX
            ),
            borrow=True
        )

        # the bias is a 1D tensor -- one bias per output feature map
        b_values = numpy.zeros((filter_shape[0],), dtype=theano.config.floatX)
        self.b = theano.shared(value=b_values, borrow=True)

        # convolve input feature maps with filters
        conv_out = conv2d(
            input=input,
            filters=self.W,
            filter_shape=filter_shape,
            image_shape=image_shape
        )

        # downsample each feature map individually, using maxpooling
        pooled_out = downsample.max_pool_2d(
            input=conv_out,
            ds=poolsize,
            ignore_border=True
        )

        # add the bias term. Since the bias is a vector (1D array), we first
        # reshape it to a tensor of shape (1, n_filters, 1, 1). Each bias will
        # thus be broadcasted across mini-batches and feature map
        # width & height
        self.output = T.tanh(pooled_out + self.b.dimshuffle('x', 0, 'x', 'x'))

        # store parameters of this layer
        self.params = [self.W, self.b]

        # keep track of model input
        self.input = input

In [3]:
learning_rate = 0.1
n_epochs = 200
nkerns = [64, 128]
batch_size = 50

In [4]:
rng = numpy.random.RandomState(23455)

x_total, y_total = preprocessing(pull=False, fine=False)
x_train, x_valid, y_train, y_valid = train_test_split(x_total, y_total, test_size=0.5, random_state=42)
max_x = max(x_train.max(), x_train.max())
x_train *= 1.0 / max_x
x_valid *= 1.0 / max_x
# y_train = boolean_y(y_train)
# y_valid = boolean_y(y_valid)

x_train = x_train.astype(float)
x_valid = x_valid.astype(float)

# compute number of minibatches for training, validation and testing
n_train_batches = x_train.shape[0]
n_valid_batches = y_train.shape[0]
n_train_batches //= batch_size
n_valid_batches //= batch_size


x_train = theano.shared(x_train.astype(theano.config.floatX))
x_valid = theano.shared(x_valid.astype(theano.config.floatX))
y_train = theano.shared(y_train)
y_valid = theano.shared(y_valid)
y_train = T.cast(y_train, 'int32')
y_valid = T.cast(y_valid, 'int32')


In [5]:

# allocate symbolic variables for the data
index = T.lscalar()  # index to a [mini]batch

# start-snippet-1
x = T.matrix('x')   # the data is presented as rasterized images
y = T.ivector('y')  # the labels are presented as 1D vector of
                    # [int] labels

In [6]:

######################
# BUILD ACTUAL MODEL #
######################
print '... building the model'

# Reshape matrix of rasterized images of shape (batch_size, 28 * 28)
# to a 4D tensor, compatible with our LeNetConvPoolLayer
# (28, 28) is the size of MNIST images.
layer0_input = x.reshape((batch_size, 1, NUM_PIXELS, NUM_PIXELS))

# Construct the first convolutional pooling layer:
# filtering reduces the image size to (28-5+1 , 28-5+1) = (24, 24)
# maxpooling reduces this further to (24/2, 24/2) = (12, 12)
# 4D output tensor is thus of shape (batch_size, nkerns[0], 12, 12)
layer0 = LeNetConvPoolLayer(
    rng,
    input=layer0_input,
    image_shape=(batch_size, 1, NUM_PIXELS, NUM_PIXELS),
    filter_shape=(nkerns[0], 1, 5, 5),
    poolsize=(2, 2)
)

# Construct the second convolutional pooling layer
# filtering reduces the image size to (12-5+1, 12-5+1) = (8, 8)
# maxpooling reduces this further to (8/2, 8/2) = (4, 4)
# 4D output tensor is thus of shape (batch_size, nkerns[1], 4, 4)
layer1 = LeNetConvPoolLayer(
    rng,
    input=layer0.output,
    image_shape=(batch_size, nkerns[0], 10, 10),
    filter_shape=(nkerns[1], nkerns[0], 5, 5),
    poolsize=(2, 2)
)

# the HiddenLayer being fully-connected, it operates on 2D matrices of
# shape (batch_size, num_pixels) (i.e matrix of rasterized images).
# This will generate a matrix of shape (batch_size, nkerns[1] * 4 * 4),
# or (500, 50 * 4 * 4) = (500, 800) with the default values.
layer2_input = layer1.output.flatten(2)

# construct a fully-connected sigmoidal layer
layer2 = HiddenLayer(
    rng,
    input=layer2_input,
    n_in=nkerns[1] * 3 * 3,
    n_out=500,
    activation=T.tanh
)

# classify the values of the fully-connected sigmoidal layer
layer3 = LogisticRegression(input=layer2.output, n_in=500, n_out=2)

# the cost we minimize during training is the NLL of the model
cost = layer3.negative_log_likelihood(y)

# create a function to compute the mistakes that are made by the model

validate_model = theano.function(
    [index],
    layer3.errors(y),
    givens={
        x: x_valid[index * batch_size: (index + 1) * batch_size],
        y: y_valid[index * batch_size: (index + 1) * batch_size]
    }
)

# create a list of all model parameters to be fit by gradient descent
params = layer3.params + layer2.params + layer1.params + layer0.params

# create a list of gradients for all model parameters
grads = T.grad(cost, params)

# train_model is a function that updates the model parameters by
# SGD Since this model has many parameters, it would be tedious to
# manually create an update rule for each model parameter. We thus
# create the updates list by automatically looping over all
# (params[i], grads[i]) pairs.
updates = [
    (param_i, param_i - learning_rate * grad_i)
    for param_i, grad_i in zip(params, grads)
]



train_model = theano.function(
    [index],
    cost,
    updates=updates,
    givens={
        x: x_train[index * batch_size: (index + 1) * batch_size],
        y: y_train[index * batch_size: (index + 1) * batch_size]
    }
)
# end-snippet-1
print "... built the model!"

... building the model
... built the model!


In [7]:

###############
# TRAIN MODEL #
###############
print '... training'
# early-stopping parameters
patience = 5000  # look as this many examples regardless
patience_increase = 2  # wait this much longer when a new best is
                       # found
improvement_threshold = 0.995  # a relative improvement of this much is
                               # considered significant
validation_frequency = min(n_train_batches, patience / 2)
                              # go through this many
                              # minibatche before checking the network
                              # on the validation set; in this case we
                              # check every epoch

best_validation_loss = numpy.inf
best_iter = 0
test_score = 0.
start_time = timeit.default_timer()

epoch = 0
done_looping = False

while (epoch < n_epochs) and (not done_looping):
    epoch = epoch + 1
    for minibatch_index in xrange(n_train_batches):

        iter = (epoch - 1) * n_train_batches + minibatch_index

        if iter % 100 == 0:
            print 'training @ iter = ', iter
        cost_ij = train_model(minibatch_index)

        if (iter + 1) % validation_frequency == 0:

            # compute zero-one loss on validation set
            validation_losses = [validate_model(i) for i
                                 in xrange(n_valid_batches - 1)]
            this_validation_loss = numpy.mean(validation_losses)
            print('epoch %i, minibatch %i/%i, validation error %f %%' %
                  (epoch, minibatch_index + 1, n_train_batches,
                   this_validation_loss * 100.))

            # if we got the best validation score until now
            if this_validation_loss < best_validation_loss:

                #improve patience if loss improvement is good enough
                if this_validation_loss < best_validation_loss *  \
                   improvement_threshold:
                    patience = max(patience, iter * patience_increase)

                # save best validation score and iteration number
                best_validation_loss = this_validation_loss
                best_iter = iter

                print(('     epoch %i, minibatch %i/%i, val error of '
                       'best model %f %%') %
                      (epoch, minibatch_index + 1, n_train_batches,
                       numpy.mean(this_validation_loss) * 100.))

        if patience <= iter:
            done_looping = True
            break

end_time = timeit.default_timer()
print('Optimization complete.')
print('Best validation score of %f %% obtained at iteration %i'%
      (best_validation_loss * 100., best_iter + 1))
print >> sys.stderr, ('The code for file ' +
                      os.path.split(__file__)[1] +
                      ' ran for %.2fm' % ((end_time - start_time) / 60.))


... training
training @ iter =  0
training @ iter =  100
epoch 1, minibatch 145/145, validation error 41.152778 %
     epoch 1, minibatch 145/145, val error of best model 41.152778 %
training @ iter =  200
epoch 2, minibatch 145/145, validation error 42.000000 %
training @ iter =  300
training @ iter =  400
epoch 3, minibatch 145/145, validation error 41.375000 %
training @ iter =  500
epoch 4, minibatch 145/145, validation error 39.013889 %
     epoch 4, minibatch 145/145, val error of best model 39.013889 %
training @ iter =  600
training @ iter =  700
epoch 5, minibatch 145/145, validation error 36.000000 %
     epoch 5, minibatch 145/145, val error of best model 36.000000 %
training @ iter =  800
epoch 6, minibatch 145/145, validation error 32.791667 %
     epoch 6, minibatch 145/145, val error of best model 32.791667 %
training @ iter =  900
training @ iter =  1000
epoch 7, minibatch 145/145, validation error 30.402778 %
     epoch 7, minibatch 145/145, val error of best model 30.

KeyboardInterrupt: 